In [1]:
import pandas as pd

In [2]:
# !conda install -c conda-forge BeautifulSoup4 --yes # 
from bs4 import BeautifulSoup # import BeautifulSoup library

In [3]:
# install a parser
# !conda install -c conda-forge html5lib --yes # 

In [4]:
# install request library
# !conda install -c conda-forge request --yes
import requests

## Scrape Neighborhood Information

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text # Get the source html code
# add on .text to get the source code from response objec

In [6]:
soup = BeautifulSoup(source, 'lxml')

In [7]:
# Inspect the soup object and identify the "table" class where the desired information is located.
table = soup.find('table')

In [8]:
rowsList = [] # append rows of table
for rows in table.find_all('tr'):
    rowsList.append(rows.text)

In [9]:
pstcd = []
Borough = []
Neighborhood = []
Neighborhood_data = {'postcode':[], 'Borough':[], 'Neighborhood':[]} # neighborhood data
# rowslist includes '\n'. 
# Split using '\n' and extract the postcode, Borough and Neighborhood text for every row.
for row in rowsList:
    pstcd.append(row.split('\n')[1])
    Borough.append(row.split('\n')[2])
    Neighborhood.append(row.split('\n')[3])

# Fill dictionary    
Neighborhood_data['postcode'] = pstcd[1:]
Neighborhood_data['Borough'] = Borough[1:]
Neighborhood_data['Neighborhood'] = Neighborhood[1:]

# Make dataframe of neighborhood data using dictionary
Neighbors = pd.DataFrame(data = Neighborhood_data)

In [10]:
Neighbors.head(5)

,postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
# Change column name
Neighbors.columns=['PostalCode', 'Borough', 'Neighborhood']
Neighbors_orig = Neighbors
Neighbors.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop the rows where Borough is "Not assigned"

In [15]:
# find index of rows where Borough is 'Not assigned'
drop_index = (Neighbors[Neighbors['Borough'].str.match('Not assigned')]).index 

# drop the rows containing 'Not assigned' in 'Borough' column
Neighbors.drop(drop_index, inplace = True)
Neighbors.reset_index(inplace = True, drop=True)


### Combine neighborhoods that share the same PostalCode

In [33]:
Neighbors = Neighbors.groupby('PostalCode').agg({'Borough':'first',
                                        'Neighborhood': ', '.join}).reset_index()

In [34]:
Neighbors.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Assign Borough name to Neighborhood name when the latter is "Not assigned"

In [42]:
# Check the cells that have a borough, but a Not assigned neighborhood. 
#Assign the Borough name to Neighborhood.

# find index of the rows where Neighborhood is "Not assigned"
tempInd = (Neighbors[Neighbors['Neighborhood'].str.match('Not assigned')]).index

# assign the Borough name to Neighborhood name
Neighbors.iloc[tempInd, 2] = Neighbors.iloc[tempInd, 1]

### 